In [5]:
import pandas as pd
import numpy as np
import os

NOTEBOOK_DIR = os.getcwd()
project_dir = os.path.dirname(NOTEBOOK_DIR)
apps_dir = os.path.join(project_dir, 'apps')
home_dir = os.path.join(apps_dir, 'home')
static_dir = os.path.join(apps_dir, 'static')
DATA_DIR = os.path.join(static_dir, 'assets', 'data')

### Load Site design Excel file and export as pickel file for faster operation

In [7]:
SITE_DESIGN_FN = os.path.join(NOTEBOOK_DIR, 'field_design.xlsm')
SITE_DESIGN = [
    ['sidebar'],
    ['page','page_element'],
    ['meta_form'],
    ['form','form_field'],
    ['model','model_field'],
    ['table','table_field']
]
SITE_DESIGN_SHEETS = ['sidebar','page','page_element','meta_form','form','form_field','model','model_field','table','table_field']
int_cols = [[],[],[],[],[],['Size'],[],[],[],[]]

def convert_cols(df, _int_cols):
    df = df.sort_values(by='#').drop('#', axis=1)
    cols = df.columns.tolist()
    str_cols = [c for c in cols if c not in _int_cols]
    dt_dct = {c:np.dtype('i1') for c in _int_cols}
    dt_dct.update({c:np.dtype('str') for c in str_cols})
    df[_int_cols] = df[_int_cols].fillna(-1)
    df[str_cols] = df[str_cols].fillna('')
    df = df.astype(dt_dct)
    return df

def export_pickles():
    for _sheet, _int_cols in zip(SITE_DESIGN_SHEETS, int_cols):
        convert_cols(
            pd.read_excel(
                SITE_DESIGN_FN,
                sheet_name=_sheet
            ),
            _int_cols
        ).to_pickle(
            os.path.join(DATA_DIR, f'{_sheet}.pkl')
        )
        print('Exported PKL: ', f'{_sheet}.pkl')
    return

export_pickles()

Exported PKL:  sidebar.pkl
Exported PKL:  page.pkl
Exported PKL:  page_element.pkl
Exported PKL:  meta_form.pkl
Exported PKL:  form.pkl
Exported PKL:  form_field.pkl
Exported PKL:  model.pkl
Exported PKL:  model_field.pkl
Exported PKL:  table.pkl
Exported PKL:  table_field.pkl


### Print models.py code generated from site design

In [3]:
def model_str():    
    export_pickles()
    code_list = ['']
    models = pd.read_pickle(os.path.join(DATA_DIR, f'model.pkl'))
    model_fields = pd.read_pickle(os.path.join(DATA_DIR, f'model_field.pkl'))
    for _, m in models.iterrows():
        fields = "\n".join([f'    {f.Code}'for _, f in model_fields[model_fields.Model == m.Model].iterrows()])
        if fields == '':
            fields = "    # No Fields Defined"
        code = "\n".join([
            f"\nclass {m.Model}({m.Extends}):",
            f"{fields}",
            f'    def __str__(self):',
            f"        return f'{m.String}'"
        ])
        code_list += [code]
    s = "\n".join(code_list)
    pd.DataFrame([s]).to_clipboard(index=False, header=False, quotechar='\t')
    return
    
model_str()

Exported PKL:  sidebar.pkl
Exported PKL:  page.pkl
Exported PKL:  page_element.pkl
Exported PKL:  meta_form.pkl
Exported PKL:  form.pkl
Exported PKL:  form_field.pkl
Exported PKL:  model.pkl
Exported PKL:  model_field.pkl
Exported PKL:  table.pkl
Exported PKL:  table_field.pkl


In [4]:
def form_str():    
    export_pickles()
    code_list = ['']
    forms = pd.read_pickle(os.path.join(DATA_DIR, f'form.pkl'))
    form_fields = pd.read_pickle(os.path.join(DATA_DIR, f'form_field.pkl'))
    for _, f in forms.iterrows():
        fields = "', '".join(form_fields[form_fields.Form == f.Form].Field.tolist())
        code = "\n".join([
            f"\nclass {f.Form}({f.Extends}):",
            f"    class Meta:",
            f"      model = {f.Model}",
            f"      fields = ['{fields}']"
        ])
        code_list += [code]
    s = "\n".join(code_list)    
    pd.DataFrame([s]).to_clipboard(index=False, header=False, quotechar='\t')
    return

form_str()

Exported PKL:  sidebar.pkl
Exported PKL:  page.pkl
Exported PKL:  page_element.pkl
Exported PKL:  meta_form.pkl
Exported PKL:  form.pkl
Exported PKL:  form_field.pkl
Exported PKL:  model.pkl
Exported PKL:  model_field.pkl
Exported PKL:  table.pkl
Exported PKL:  table_field.pkl


In [ ]:
MANIFEST_HEADING_FIELDS = [
    ('Amazon 1', {'item description' : 'description', 'gl description' : 'department', 'product class' : 'class', 'category' : 'category', 'subcategory' : 'subcategory', 'qty' : 'units', 'unit retail' : 'unit_retail', 'asin' : '', 'ext. retail' : '', 'inventory reference id' : '', 'shipmentid' : '', 'shipmentitemid' : '', 'shipped qty' : '', 'unfulfilled qty' : '', 'upc' : ''}),
    ('B-Stock 1', {'item description' : 'description', 'manufacturer' : 'brand', 'model' : 'model', 'qty' : 'units', 'retail per unit' : 'unit_retail', 'carrier' : '', 'expiration date' : '', 'memory' : '', 'total retail' : '', 'upc' : ''}),
    ('Costco 1', {'item description' : 'description', 'department' : 'department', 'category' : 'category', 'brand' : 'brand', 'qty' : 'units', 'unit retail' : 'unit_retail', 'category code' : '', 'dept. code' : '', 'ext. retail' : '', 'item #' : '', 'lot #' : ''}),
    ('Home Depot 1', {'item description' : 'description', 'department' : 'department', 'product class' : 'class', 'category' : 'category', 'subcategory' : 'subcategory', 'brand' : 'brand', 'model' : 'model', 'qty' : 'units', 'unit retail' : 'unit_retail', 'ext. retail' : '', 'order #' : '', 'sb #' : '', 'sku' : '', 'upc' : ''}),
    ('Target 1', {'item description' : 'description', 'department' : 'department', 'product class' : 'class', 'category' : 'category', 'subcategory' : 'subcategory', 'brand' : 'brand', 'qty' : 'units', 'retail' : 'unit_retail', 'item' : '', 'ext. retail' : '', 'pallet id' : '', 'upc' : ''}),
    ('Target 2', {'item description' : 'description', 'sep cat' : 'department', 'product type' : 'class', 'category' : 'category', 'item model' : 'model', 'qty' : 'units', 'retail' : 'unit_retail', 'cat.' : '', 'cat. desc.' : '', 'container type' : '', 'ext. retail' : '', 'item' : '', 'item dep id' : '', 'origin' : '', 'pallet' : '', 'warehouse code' : ''}),
    ('Walmart 1', {'item description' : 'description', 'department' : 'department', 'subcategory' : 'category', 'model' : 'model', 'qty' : 'units', 'unit retail' : 'unit_retail', 'upc' : '', 'ext. retail' : '', 'unit weight' : '', 'pallet id' : '', 'pallet name' : '', 'pallet type' : ''})
]

def GET_MANIFEST_FIELDS(field_list):
    heading_fields, match_cnt, match_pct, heading_template = {}, 0, 0, ''
    for _d, h in MANIFEST_HEADING_FIELDS:
        _match_cnt = len([1 for f in field_list if f.lower() in h.keys()])
        _match_pct = _match_cnt / len(h.keys())
        replace = _match_cnt > match_cnt
        if _match_cnt == match_cnt: replace = _match_pct > match_pct
        if replace:
            heading_fields, match_cnt, match_pct, heading_template = h, _match_cnt, _match_pct, _d
    for k in [k for k, v in heading_fields.items() if v == '']: del heading_fields[k] 
    return heading_template, heading_fields

In [ ]:
from datetime import date, datetime, timedelta
from string import ascii_uppercase

ECO_OPEN_DATE = date(2022, 10, 31)
ECO_PRICE_COUNT = 104
ECO_PRICES = np.array([    
    1,     2,     3,     4,     5,     6,     7,     8,     9,
    10,    11,    12,    14,    16,    18,    20,    23,    26,
    29,    32,    35,    38,    41,    44,    47,    50,    55,
    60,    65,    70,    75,    80,    85,    90,    95,   100,
    110,   120,   140,   160,   180,   200,   230,   260,   290,
    320,   350,   380,   410,   440,   470,   500,   550,   600,
    650,   700,   750,   800,   850,   900,   950,  1000,  1100,
    1200,  1400,  1600,  1800,  2000,  2300,  2600,  2900,  3200,
    3500,  3800,  4100,  4400,  4700,  5000,  5500,  6000,  6500,
    7000,  7500,  8000,  8500,  9000,  9500, 10000, 11000, 12000,
    14000, 16000, 18000, 20000, 23000, 26000, 29000, 32000, 35000,
    38000, 41000, 44000, 47000, 50000
])

def GET_TODAYS_PRICES():
    _dict, _reverse_dict = {}, {}
    _prices = np.sort(ECO_PRICES)[::-1]
    delta_days = (date.today() - ECO_OPEN_DATE).days
    price_delta = delta_days // 7
    
    for i in range(1,5):
        for c in ascii_uppercase:
            price_delta = price_delta % ECO_PRICE_COUNT
            lbl, price = f'{c}{i}', ECO_PRICES[price_delta]  
            _dict[lbl] = price
            _reverse_dict[price] = lbl
            price_delta += 1
    
    return _dict, _reverse_dict

PRICE_DICT, REVERSE_PRICE_DICT = GET_TODAYS_PRICES()

def GET_PRICE_ARRAY(from_price):
    _i = np.searchsorted(ECO_PRICES, from_price)
    return np.concatenate([ECO_PRICES[_i:], ECO_PRICES[:_i]])


In [ ]:
loc_objects = [
{'section':{'name': 'c'}, 'label':'thisA'},
{'section':{'name': 'b'}, 'label':'thisA'},
{'section':{'name': 'a'}, 'label':'thisC'},
]

loc = np.array([[l['section']['name'], l['label']] for l in loc_objects])
loc = loc[loc[:,0].argsort()]
[(f'{l[0]}_{l[1]}', f'{l[0]} - {l[1]}') for l in loc]

In [ ]:
from blabel import LabelWriter

label_writer = LabelWriter("item_template.html",
                           default_stylesheets=("style.css",))
records= [
    dict(sample_id="s01", sample_name="Sample 1"),
    dict(sample_id="s02", sample_name="Sample 2")
]

label_writer.write_labels(records, target='qrcode_and_label.pdf')

In [ ]:
# import the modules
import sys
import time

from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler


# to trigger the proess_and_plt function when a new file comes in directory 

class ExampleHandler(FileSystemEventHandler):
    def on_created(self, event):
        print(event.src_path)

event_handler = ExampleHandler()

observer = Observer()
observer.schedule(event_handler, path='C:\\Users\\bill_\\Downloads')

observer.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()

observer.join()

In [ ]:
import os
import sys

import win32con as wcon
import win32print as wprn
import win32ui as wui
from PIL import Image as pil_image, ImageWin as pil_image_win


def add_txt(hdc, file_name, new_page=False):
    if new_page:
        hdc.StartPage()
    pixel_scale = 84
    with open(file_name, mode="r") as fin:
        for idx, line in enumerate(fin):
            print("Text line {:d}: {:s}".format(idx, line))
            hdc.TextOut(5, idx * pixel_scale, line)
    if new_page:
        hdc.EndPage()


def draw_img(hdc, dib, maxh, maxw):
    w, h = dib.size
    print("Image HW: ({:d}, {:d}), Max HW: ({:d}, {:d})".format(h, w, maxh, maxw))
    h = min(h, maxh)
    w = min(w, maxw)
    l = (maxw - w) // 2
    t = (maxh - h) // 2
    dib.draw(hdc, (l, t, l + w, t + h))


def add_img(hdc, file_name, new_page=False):
    if new_page:
        hdc.StartPage()
    maxw = hdc.GetDeviceCaps(wcon.HORZRES)
    maxh = hdc.GetDeviceCaps(wcon.VERTRES)
    img = pil_image.open(file_name)
    dib = pil_image_win.Dib(img)
    draw_img(hdc.GetHandleOutput(), dib, maxh, maxw)
    if new_page:
        hdc.EndPage()


def main():
    printer_name = "Rollo Printer"  #wprn.GetDefaultPrinter()
    
    #out_file = ".\\test2.pdf"
    out_file = None  # Send to printer directly
    hdc = wui.CreateDC()
    hdc.CreatePrinterDC(printer_name)
    if out_file and os.path.isfile(out_file):
        os.unlink(out_file)
    hdc.StartDoc("Test", out_file)

    add_txt(hdc, "google account.txt")
    add_img(hdc, "some_file.png")

    hdc.EndDoc()
    hdc.DeleteDC()

In [ ]:
hdc.GetTextFace()

In [ ]:
hdc.c

In [ ]:
from pdf_reports import JupyterPDF

JupyterPDF("path_to_some.pdf")

In [ ]:
printer_name = "Rollo Printer"
file_name = "some_file.png"

out_file = None
hdc = wui.CreateDC()
hdc.CreatePrinterDC(printer_name)

hdc.StartDoc("Test", out_file)

img = pil_image.open(file_name)
dib = pil_image_win.Dib(img)

dib.draw(hdc.GetHandleOutput(), (250, 50, 400, 200))

hdc.TextOut(10, 10, 'Description')
hdc.TextOut(300, 10, 'A1')
hdc.TextOut(10, 55, 'Brand')
hdc.TextOut(10, 100, 'Model')
hdc.TextOut(10, 145, 'Condition')

hdc.EndDoc()
hdc.DeleteDC()

In [ ]:
print("Python {:s} {:03d}bit on {:s}\n".format(" ".join(elem.strip() for elem in sys.version.split("\n")),
                                                64 if sys.maxsize > 0x100000000 else 32, sys.platform))
main()
print("\nDone.")

In [ ]:
import qrcode
img = qrcode.make(132654687)
type(img)  # qrcode.image.pil.PilImage
img.save("some_file.png")

In [ ]:
from PIL import Image, ImageDraw, ImageFont

# get an image
with Image.open("some_file.png").convert("RGBA") as base:
    color = (0, 0, 0, 255)

    # make a blank image for the text, initialized to transparent text color
    txt = Image.new("RGBA", (290, 290), (255, 255, 255, 122))
    # get a font
    fnt = ImageFont.truetype("FreeMono.ttf", 40)

    # get a drawing context
    d = ImageDraw.Draw(txt)

    # draw text, half opacity
    d.text((10, 10), "Hello", font=fnt, fill=color)
    # draw text, full opacity
    d.text((10, 60), "World", font=fnt, fill=color)

    out = Image.alpha_composite(base, txt)

    out.show()

In [9]:
str = 'a_dct[name] is b_dct[years] b_dct[title]'



'a_dct[name] is b_dct[years] b_dct[title]'

In [22]:
def DICTS_FORMAT(str, dicts, cycles=3):
    for z in range(cycles):
        for dict_name, dict_kvs in dicts.items():        
            for k,v in dict_kvs.items(): 
                str = str.replace(f'{dict_name}[{k}]', f'{v}')
    return str

str = 'a_dct[name] is b_dct[years] b_dct[title]'

dicts = {
'a_dct' : {'name':'b_dct[years]'},
'b_dct' : {'years':'a_dct[name]', 'title': 'young'}
}


In [57]:
import numpy as np

thisl = ['<div>l</div>']
thisc = []
thisr = ['<div>r</div>']






<div class="row">
<div class="col-6 text-left">
<div>l</div>
</div>
<div class="col-6 text-right">
<div>r</div>
</div>
</div>


In [45]:
print("\n".join(['a','b','c']))

a
b
c
